In [ ]:
def volatility_parity(dataframe, target=0.03, notional=100, window=5, leverage_cap=2):
    """
    Generate a volatility targeting investment amount (weight) for each asset per day based on a self-defined target volatility and a self-defined rolling window.

    :param dataframe: pd.dataframe, a panel dataframe with several assets' daily returns over an arbitrary period.
    :param target: float64 > 0, self-defined annualized volatility target for each asset. (0.03 by default)
    :param notional: integer > 0, the notional amount that we originally allocate in each trade. (100 dollars by default)
    :param window: integer > 0, the rolling window (days) for calculating the annualized standard deviation (volatility). (5 days by default)
    :param leverage_cap: float64 >= 0.5,  the leverage cap applied to limit the scaled weights for assets. (2x by default)
    :return: a dataframe with the notional amount being volatility-adjusted for each asset per day.
    """

    frame = []
    dataframe_1 = pd.DataFrame()
    indexes = list(dataframe.columns)
    for index in indexes:
        vol = dataframe[index].rolling(window=window).std() * np.sqrt(252/window)
        vol_scaled_weight = (target/vol) * notional
        frame.append(vol_scaled_weight.to_frame(index))
        dataframe_1 = pd.concat(frame, axis=1)
        dataframe_1[dataframe_1 > leverage_cap * notional] = leverage_cap * notional
        dataframe_1[dataframe_1 < -leverage_cap * notional] = -leverage_cap * notional
    return dataframe_1.shift(1)